# [NTDS'18] milestone 4: graph signal processing
[ntds'18]: https://github.com/mdeff/ntds_2018

[Rodrigo Pena](https://people.epfl.ch/254838?lang=en)

[EPFL LTS2](http://lts2.epfl.ch)

The goal of this milestone is to do some Graph Signal Processing (GSP) on the data of your project. We will use the [PyGSP](https://github.com/epfl-lts2/pygsp) library to help us do that. If you have not yet done so, install PyGSP with
```sh
pip install pygsp
```
or
```sh
conda install -c conda-forge pygsp
```

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import sparse, spatial
from pygsp import graphs, filters, plotting

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (17, 5)
plotting.BACKEND = 'matplotlib'

## 1 - Preprocessing
- Work with a subset of graph and labels if your dataset is too big.
- Load network as a PyGSP graph. [Documentation](https://pygsp.readthedocs.io/en/latest/reference/graphs.html)
- Compute Laplacian (normalized? combinatorial?)

In [ ]:
# Your code goes here
# BEGIN


# END

**Answer:** [Your answer goes here]

## 2 - Graph Fourier Transform
- Compute Fourier basis (G.e, G.U)
- Visualize distribution of eigenvalues
- Visualize graph with laplacian eigenmaps (k = 2 or 3) (G.set_coordinates)
- Visualize a couple of eigenvectors on the spectral embedding (a couple of low frequency ones, and a couple of high-frequency ones)
- Visualize some label signal on the graph
- Plot the GFT of this label signal.
- Discuss amplitude of GFT vis-a-vis similarity between label signal and eigenvectors.
- Diffusion on graphs

In [ ]:
# Your code goes here
# BEGIN


# END

**Answer:** [Your answer goes here]

## 3 - Diffusion on graphs
- Heat kernel (f = filters.Heat(G, tau=10); f.plot())
- Localization of filtered Diracs

In [ ]:
# Your code goes here
# BEGIN


# END

**Answer:** [Your answer goes here]

## 4 - De-noising 
- $\text{arg} \, \underset{f\in \mathbb{R}^{n}}{\min} \|f - y\|^2 + \gamma f^{\top} L f$
- Derive closed form solution as a graph filtering operation. Implementation with `np.linalg.solve()`.
- Add noise to label and try to recover it (compare heavy-tailed vs. Gaussian noise?)
- Plot localization of filtered Diracs and compare with heat diffusion
- Compare GFT of noisy and recovered signal. What do you observe?

In [ ]:
# Your code goes here
# BEGIN


# END

**Answer:** [Your answer goes here]

## 5 - Transductive learning
- Subsample your label signal.
- Simple label propagation.
- TL by minimizing quadratic form. Then, find a threshold.
- TL by minimizing graph total variation?
    - Relationship with graph cuts
- Compare those 3 approaches.
    - Error and std. dev. as function of regularization parameter.
    - Prediction error as function of m/n for the best regularization parameter.

In [ ]:
# Your code goes here
# BEGIN


# END

**Answer:** [Your answer goes here]

## 6 - Takeaways

**Answer:** [Your answer goes here]